# read in standard creds

[![Tutorial Video pt 1 of 4]](https://youtu.be/7aTHw7scsa8)

In [ ]:
import os
import json

from urllib.parse import urlparse
import datetime as dt
from dateutil.parser import parse

from dataclasses import dataclass, field

from pprint import pprint

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.get_data as gd
import domolibrary.utils.DictDot as util_dd

In [ ]:
# |eval: false

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_password=os.environ["DOJO_PASSWORD"],
    domo_username="jae@onyxreporting.com",
)

await full_auth.print_is_token()

# full_auth.generate_auth_header(token = full_auth.token)

🎉 token retrieved from domo-community ⚙️


True

## adding jupyter auth creds

In [ ]:
# |eval: false

USER_TOKEN = "XQq01p8JGm7GLvN8c5BaZw4hPnYbpf"  # pull from network traffic
SERVICE_LOCATION = (
    "jupyter-prod1.domodatascience.com"
    or urlparse(os.environ["JUPYTERHUB_SERVICES_URL"]).netloc
)  # pull from domo
SERVICE_PREFIX = (
    "/user/domo-community-1893952720/19fb3535/"
    or os.environ["JUPYTERHUB_SERVICE_PREFIX"]
)

user_token = USER_TOKEN
# input("user token:")
# retrieve this by monitoring domo jupyter network traffic.  it is the token header

service_location = SERVICE_LOCATION
# input("service_location")
# retrieve from domo jupyter env

service_prefix = SERVICE_PREFIX
# input("service prefix") or
# retrieve from domo jupyter env

print(user_token, service_location, service_prefix)

XQq01p8JGm7GLvN8c5BaZw4hPnYbpf jupyter-prod1.domodatascience.com /user/domo-community-1893952720/19fb3535/


# Library Extensions
## CLIENT
### define DomoJupyterAuth class

extend `DomoFullAuth` with new properties and replace an existing method


In [ ]:
# |eval: false


@dataclass
class _DomoJupyter_Optional:
    def __post_init__(self):

        self.user_token = self.user_token or input(
            "user token: # retrieve this by monitoring domo jupyter network traffic.  it is the token header"
        )
        self.service_location = self.service_location or input(
            "service_location:  # retrieve from domo jupyter env"
        )
        self.service_prefix = self.service_prefix or input(
            "service prefix: # retrieve from domo jupyter env"
        )

        self._test_prereq()

    def generate_auth_header(self, token: str) -> dict:
        self.auth_header = {
            "x-domo-authentication": token,
            "authorization": f"Token {self.user_token}",
        }

        return self.auth_header


@dataclass
class _DomoJupyter_Required:
    user_token: str
    service_location: str
    service_prefix: str

    def get_user_token_flow(self):
        """stub"""
        print("hello world i am a user_token_flow")

    def _test_prereq(self):
        if not self.user_token:
            raise Exception("DomoJupyterAuth objects must have a user_token")

        if not self.service_location:
            raise Exception("DomoJupyterAuth objects must have a service_location")

        if not self.service_prefix:
            raise Exception("DomoJupyterAuth objects must have a service_prefix")

        if not self.user_token or not self.service_location or not self.service_prefix:
            raise Exception(
                "DomoJupyterAuth objects must have user_token, service_location and service_prefix"
            )


@dataclass
class DomoJupyterFullAuth(
    _DomoJupyter_Optional, dmda.DomoFullAuth, _DomoJupyter_Required
):
    @classmethod
    def convert_auth(
        cls, full_auth: dmda.DomoFullAuth, user_token, service_location, service_prefix
    ):
        """converts dmda.DomoFullAuth to DomoJupyterFullAuth
        i.e. adds DomoJupyter specific auth fields
        eventually can add DomoJupyter specific auth flow for generating auth token
        """
        return cls(
            domo_instance=full_auth.domo_instance,
            domo_username=full_auth.domo_username,
            domo_password=full_auth.domo_password,
            user_token=user_token,
            service_location=service_location,
            service_prefix=service_prefix,
        )


dj_auth = DomoJupyterFullAuth.convert_auth(
    full_auth=full_auth,
    user_token=user_token,
    service_location=service_location,
    # service_prefix=None
    service_prefix=service_prefix,
)

# token = await dj_auth.get_auth_token()
# dj_auth.generate_auth_header( token = token)

## ROUTES

[![Tutorial Video pt 2 of 4]](https://youtu.be/7aTHw7scsa8)

### GET Routes


In [ ]:
# |eval: false


async def _get_jupyter_content(
    auth: dmda.DomoAuth,
    content_path: str = "",
    debug_api: bool = False,
):
    url = f"https://{auth.domo_instance}.{auth.service_location}{auth.service_prefix}api/contents/{content_path}"

    res = await gd.get_data(
        url=f"{url}",
        method="GET",
        auth=auth,
        headers={"authorization": f"Token {auth.user_token}"},
        debug_api=debug_api,
        num_stacks_to_drop=1,
    )
    if not res.is_success:
        raise Exception("unable to retrieve content")

    return res

#### sample implementation of get_jupyter_content

In [ ]:
# | eval: false

res = await _get_jupyter_content(
    auth=dj_auth,
    # content_path = 'my_terrible_demo.txt',
    # content_path="datatypes.ipynb",
    debug_api=False,
)

res.response["content"][0:2]

[{'name': 'my_terrible_demo.txt',
  'path': 'my_terrible_demo.txt',
  'last_modified': '2023-09-15T22:20:36.711000Z',
  'created': '2023-09-15T22:20:36.711000Z',
  'content': None,
  'format': None,
  'mimetype': 'text/plain',
  'size': 33,
  'writable': True,
  'type': 'file'},
 {'name': 'untitled1.txt',
  'path': 'untitled1.txt',
  'last_modified': '2023-09-15T22:37:23.077000Z',
  'created': '2023-09-15T22:37:23.077000Z',
  'content': None,
  'format': None,
  'mimetype': 'text/plain',
  'size': 33,
  'writable': True,
  'type': 'file'}]

### update jupyter content

improve code maintenance by implementing a factory design patternx

In [ ]:
# | eval: false


def _generate_update_jupyter_text(body):
    body.update(
        {
            "format": "text",
            "type": "file",
        }
    )
    return body


def _generate_update_jupyter_ipynb(body):
    body.update(
        {
            "format": None,
            "type": "notebook",
        }
    )
    return body


def _generate_update_jupyter_directory(content_path, body):

    if "/" in content_path:
        new_content_path = "/".join(content_path.split("/")[:-1])
    else:
        new_content_path = ""

    body.update(
        {
            "path": new_content_path,
            "format": None,
            "type": "directory",
        }
    )
    return body


def _generate_update_jupyter_body(
    new_content, content_path: str  # my_folder/datatypes.ipynb
):
    """factory to construct properly formed body"""

    content_name = os.path.normpath(content_path).split(os.sep)[-1]

    if "." in content_path:
        content_type = content_path.split(".")[-1]
    else:
        content_type = "directory"

    body = {
        "name": content_name,
        "content": new_content,
        "path": content_path,
    }

    if content_type == "ipynb":
        return _generate_update_jupyter_ipynb(body)

    if content_type == "directory":
        return _generate_update_jupyter_directory(content_path, body)

    return _generate_update_jupyter_text(body)


_generate_update_jupyter_body("hello world", "hi.md")

{'name': 'hi.md',
 'content': 'hello world',
 'path': 'hi.md',
 'format': 'text',
 'type': 'file'}

In [ ]:
# | eval: false


async def _update_jupyter_file(
    auth: DomoJupyterFullAuth,
    new_content,
    content_path: str = "",  # file name and location in jupyter
    debug_api: bool = False,
    num_stacks_to_drop=1,
):
    if not isinstance(auth, DomoJupyterFullAuth):
        raise Exception(
            f"invalid auth type {auth.__class__.__name__} must pass DomoJupyter auth type"
        )

    body = _generate_update_jupyter_body(new_content, content_path)

    content_path_split = os.path.normpath(content_path).split(os.sep)

    url = f"https://{auth.domo_instance}.{auth.service_location}{auth.service_prefix}api/contents/{'/'.join(content_path_split)}"

    res = await gd.get_data(
        url=f"{url}",
        method="PUT",
        auth=auth,
        body=body,
        debug_api=debug_api,
        num_stacks_to_drop=num_stacks_to_drop,
    )

    if not res.is_success:
        raise Exception("unable to update content.  validate your user token")

    return res

#### sample implementation of update_content

In [ ]:
# | eval: false

res = await _update_jupyter_file(
    auth=dj_auth,
    content_path="new_folder/my_great_demo.txt",
    new_content="jae is excellent at demoes",
    debug_api=False,
)

res.response

{'name': 'my_great_demo.txt',
 'path': 'new_folder/my_great_demo.txt',
 'last_modified': '2023-09-18T20:48:47.644000Z',
 'created': '2023-09-18T20:48:47.644000Z',
 'content': None,
 'format': None,
 'mimetype': 'text/plain',
 'size': 26,
 'writable': True,
 'type': 'file'}

### recursive get_content

[![Tutorial Video pt 3 of 4]](https://youtu.be/l32qTj08bCY)


In [ ]:
# | eval: false


async def get_content_recursive(
    auth: DomoJupyterFullAuth,
    content_path="",
    all_rows=None,
    debug_api: bool = False,
    return_raw: bool = False,
):
    all_rows = all_rows or []

    res = await _get_jupyter_content(
        auth=auth,
        content_path=content_path,
        debug_api=debug_api,
    )

    content_ls = res.response["content"]

    for obj in content_ls:
        content_path = obj["path"]

        if obj["type"] != "directory":
            res = await _get_jupyter_content(
                auth=auth,
                content_path=content_path,
                debug_api=debug_api,
            )

            all_rows.append(res.response)

        elif obj["type"] == "directory":
            await get_content_recursive(
                auth=auth,
                content_path=content_path,
                all_rows=all_rows,
                debug_api=debug_api,
            )

    if return_raw:
        return res

    res.response = all_rows

    return res

In [ ]:
# | eval: false

x = ["this will work"]  # a1


def my_fake_recursion(x=None):
    x = x or []  # a1
    x.append("new_record")  # a1

    print("i am in a function i have been passed by reference", x)


my_fake_recursion(x=x)

x  # a1

i am in a function i have been passed by reference ['this will work', 'new_record']


['this will work', 'new_record']

#### sample implementation of get_content_recursive

In [ ]:
# | eval: false

res = await _get_jupyter_content(auth=dj_auth, content_path="")
[content["name"] for content in res.response["content"]]

['my_terrible_demo.txt',
 'untitled1.txt',
 'tutorial',
 'tutorial_env.ipynb',
 'UpdateDatasets.ipynb',
 'export',
 'MakeDomoFaster.ipynb',
 'get_content.ipynb',
 'DataflowExecution.ipynb',
 'Untitled.ipynb',
 'instance_pdp_access.ipynb',
 'Untitled Folder',
 'new_folder',
 'README.md',
 'recent_executions',
 'untitled.txt',
 'env.txt',
 'datatypes.ipynb']

In [ ]:
# |eval: false

res = await get_content_recursive(
    auth=dj_auth,
    debug_api=False,
    return_raw=False,
)
content_ls = res.response

[content["name"] for content in content_ls]

['my_terrible_demo.txt',
 'untitled1.txt',
 'copy_me.ipynb',
 'AS_rbd.ipynb',
 'env.txt',
 'JW_session5_prep.ipynb',
 'tutorial_env.ipynb',
 'UpdateDatasets.ipynb',
 'untitled1.txt',
 'MakeDomoFaster.ipynb',
 'get_content.ipynb',
 'DataflowExecution.ipynb',
 'Untitled.ipynb',
 'instance_pdp_access.ipynb',
 'my_great_demo.txt',
 'README.md',
 'stderr',
 'output.ipynb',
 'stdout',
 'stderr',
 'output.ipynb',
 'stdout',
 'stderr',
 'output.ipynb',
 'stdout',
 'stderr',
 'output.ipynb',
 'stdout',
 'stderr',
 'output.html',
 'output.ipynb',
 'output.pdf',
 'stdout',
 'stderr',
 'output.ipynb',
 'stdout',
 'stderr',
 'output.ipynb',
 'stdout',
 'stderr',
 'output.ipynb',
 'stdout',
 'stderr',
 'output.ipynb',
 'stdout',
 'stderr',
 'output.ipynb',
 'stdout',
 'stderr',
 'output.ipynb',
 'stdout',
 'untitled.txt',
 'env.txt',
 'datatypes.ipynb']

## CLASS
[![Tutorial Video pt 4 of 4]](https://youtu.be/RbfbQ8V2erM)


## DomoJupyter Content and Class

In [ ]:
# |eval: false


@dataclass
class DomoJupyter_Content:
    name: str
    folder: str
    last_modified: dt.datetime
    file_type: str
    content: str

    auth: DomoJupyterFullAuth = field(repr=False)

    default_export_folder: str = "export"

    def __post_init__(self):
        if self.folder.endswith(self.name):
            self.folder = self.folder.replace(self.name, "")

    @classmethod
    def _from_json(cls, obj: dict, auth: DomoJupyterFullAuth):
        dd = util_dd.DictDot(obj) if not isinstance(obj, util_dd.DictDot) else obj

        dc = cls(
            name=dd.name,
            folder=dd.path,
            last_modified=parse(dd.last_modified),
            file_type=dd.type,
            auth=auth,
            content=obj.get("content"),
        )

        return dc

    def export_content(
        self,
        output_folder: str = None,
        file_name: str = None,
    ):
        output_folder = output_folder or os.path.join(
            self.default_export_folder, self.folder
        )

        file_name = file_name or self.name

        if not os.path.exists(output_folder):
            print(output_folder)
            os.makedirs(output_folder)

        content_str = self.content
        if isinstance(self.content, dict):
            content_str = json.dumps(self.content)

        output_path = os.path.join(output_folder, file_name)
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(content_str)
            f.close()

        return output_path

    async def update(
        self,
        jupyter_folder: str = None,
        jupyter_file_name: str = None,
        debug_api: bool = False,
    ):

        if jupyter_folder and jupyter_file_name:
            content_patuh = f"{jupyter_folder}/{jupyter_file_name}"

        if len(self.folder) > 0:
            content_path = f"{self.folder}/{self.name}"

        else:
            content_path = self.name

            if content_path.lower().startswith(self.default_export_folder.lower()):
                content_path = content_path.replace(self.default_export_folder, "")

        content_path = "/".join(os.path.normpath(content_path).split(os.sep))

        return await _update_jupyter_file(
            auth=self.auth,
            content_path=content_path,
            new_content=self.content,
            debug_api=debug_api,
        )


@dataclass
class DomoJupyter:
    auth: DomoJupyterFullAuth = field(repr=False)
    content: [DomoJupyter_Content] = field(default=None)

    user_token = None
    service_location = None
    service_prefix = None

    def __post_init__(self):
        if hasattr(self.auth, "domo_password") and not isinstance(
            self.auth, DomoJupyterFullAuth
        ):

            self.auth = DomoJupyterFullAuth.convert_auth(
                full_auth=self.auth,
                user_token=self.user_token,
                service_location=self.service_location,
                service_prefix=self.service_prefix,
            )
        if hasattr(self.auth, "developer_token") and not isinstance(
            self.auth, DomoJupyterDeveloperToken
        ):
            raise Exception(
                "this authentitcation conversion method hasn't been implemented yet"
            )

            # self.auth = DomoJupyterTokenAuth.convert_auth(
            #     auth=self.auth,
            #     user_token=self.user_token,
            #     service_location=self.service_location,
            #     service_prefix=self.service_prefix,
            # )

    async def get_content(
        self,
        debug_api: bool = False,
        return_raw: bool = False,
        is_recursive: bool = True,
        content_path: str = "",
    ):

        if is_recursive:
            res = await get_content_recursive(
                auth=self.auth, debug_api=False, content_path=content_path
            )
            content_ls = res.response

        else:
            res = await _get_jupyter_content(
                auth=self.auth, debug_api=False, content_path=content_path
            )

            content_ls = res.response["content"]

        if return_raw:
            return res

        return [
            DomoJupyter_Content._from_json(obj, auth=self.auth) for obj in content_ls
        ]

#### sample impelemntation of get_contet

In [ ]:
# |eval: false

domo_dj = DomoJupyter(auth=dj_auth)

domo_dj_content = await domo_dj.get_content(
    debug_api=False, return_raw=False, is_recursive=False
)

[domo_content.name for domo_content in domo_dj_content]

['my_terrible_demo.txt',
 'untitled1.txt',
 'tutorial',
 'tutorial_env.ipynb',
 'UpdateDatasets.ipynb',
 'export',
 'MakeDomoFaster.ipynb',
 'get_content.ipynb',
 'DataflowExecution.ipynb',
 'Untitled.ipynb',
 'instance_pdp_access.ipynb',
 'Untitled Folder',
 'new_folder',
 'README.md',
 'recent_executions',
 'untitled.txt',
 'env.txt',
 'datatypes.ipynb']

#### sample implementation of updating content with DomoJupyter_Content class

In [ ]:
# |eval: false

test_content = domo_dj_content[1]
print(test_content.__class__.__name__)

test_content.content = "jae rocks at debugging on the fly"
test_content

await test_content.update(debug_api=False)

DomoJupyter_Content


ResponseGetData(status=200, response={'name': 'untitled1.txt', 'path': 'untitled1.txt', 'last_modified': '2023-09-18T20:49:10.472000Z', 'created': '2023-09-18T20:49:10.472000Z', 'content': None, 'format': None, 'mimetype': 'text/plain', 'size': 33, 'writable': True, 'type': 'file'}, is_success=True, parent_class=None, traceback_details=TracebackDetails(function_name='_update_jupyter_file', file_name='/tmp/ipykernel_4630/2378948045.py', function_trail='<module> -> update -> _update_jupyter_file', traceback_stack=[<FrameSummary file /tmp/ipykernel_4630/64093016.py, line 9 in <module>>, <FrameSummary file /tmp/ipykernel_4630/3496755649.py, line 83 in update>, <FrameSummary file /tmp/ipykernel_4630/2378948045.py, line 22 in _update_jupyter_file>], parent_class=None))

In [ ]:
# test_content.export_content()